# Предсказание победителя в турнирной игре по кс:го

In [1]:
import os, tqdm, json, pickle, gc, itertools
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict

import catboost as cb
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, ParameterGrid, StratifiedKFold, cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance

import optuna
from optuna.samplers import TPESampler

def get_cb_scores(best_params, X_best_tr, X_best_te, y_tr, y_te):
    
    cat_features = np.argwhere(X_best_tr.dtypes.values=='object').flatten()
    best_params['cat_features'] = cat_features
    model = cb.CatBoostClassifier(**best_params)
    
    min_cv_score= cross_val_score(model,X_best_tr, y_tr, cv=CV,scoring= SCORING, n_jobs=-1).min()
    model.fit(X_best_tr, y_tr)
    te_score= roc_auc_score(y_te, model.predict_proba(X_best_te)[:, 1])
    
    return {'min_cv_score':min_cv_score, 'te_score':te_score}

class CatBoostFeatureSelector():    
    
    def __init__(self, cv, scoring, params, n_iterations, seed):
        
        self.cv = cv
        self.scoring = scoring
        self.params = params
        self.n_iterations = n_iterations
        self.seed = seed
        
    def select_features(self, X, y, cv, scoring, params, seed):

        X_tr, X_ho, y_tr, y_ho = train_test_split(X, y, test_size = .1, shuffle = True, random_state =self.seed)

        i = 1
        while True: 

            print('> n_features = {}'.format(X_tr.shape[1]))

            cat_features = np.argwhere(X_tr.dtypes.values=='object').flatten()

            if i ==1:
                verbose = 1
            else:
                verbose = 0

            new_params= {'cat_features':cat_features, 'random_state':seed, 'verbose':verbose}
            if params is not None:
                new_params.update(params) 

            model = cb.CatBoostClassifier(**new_params)        
            model.fit(X_tr, y_tr, eval_set=(X_ho, y_ho), early_stopping_rounds=10)

            mask = model.feature_importances_>0

            if np.all(mask):
                L_cv_score = cross_val_score(model, X_tr, y_tr,
                                        scoring = scoring,
                                        cv = cv,
                                        n_jobs=-1).tolist()
                print('> cv_scores: {}'.format(L_cv_score))
                break
            else:            
                X_tr, X_ho = X_tr.loc[:, mask], X_ho.loc[:, mask]

                cat_features = np.argwhere(X_tr.dtypes.values=='object').flatten()
                model = cb.CatBoostClassifier(iterations = model.best_iteration_,
                                          cat_features = cat_features,
                                          random_state = seed,
                                          verbose=verbose)

                L_cv_score = cross_val_score(model, X_tr, y_tr,
                                        scoring = scoring,
                                        cv = cv,
                                        n_jobs=-1).tolist()
                print('> cv_scores: {}'.format(L_cv_score))
                i+=1

        best_features = X_tr.columns    
        del X_tr, X_ho, y_tr, y_ho
        gc.collect()

        return {'features':best_features, 'cat_features':cat_features,'best_iter':model.best_iteration_}

    def apply_perm_imp(self, d_sel_res, X_tr, y_tr, scoring, params, n_iterations, seed):

        features = d_sel_res['features']
        cat_features = d_sel_res['cat_features']
        iterations = d_sel_res['best_iter']
        
        new_params = params.copy()
        new_params['cat_features'] = cat_features
        new_params['iterations'] = iterations

        X_tr_c=  X_tr.copy()
        X_tr_c = X_tr_c[features]
        idx_split = int(np.rint(.8*X_tr_c.shape[0]))

        model = cb.CatBoostClassifier(**new_params)
        model.fit(X_tr_c.iloc[:idx_split], y_tr.iloc[:idx_split])

        L_imps= []
        for i in tqdm.tqdm_notebook(range(n_iterations)):
            d_perm_imp = permutation_importance(model,
                                                X_tr_c.iloc[idx_split:], y_tr.iloc[idx_split:],
                                                scoring =scoring,
                                                n_repeats = 1,
                                                random_state=seed+i,
                                                n_jobs=-1)
            L_imps.append(d_perm_imp['importances'].flatten())

        arr_imp=np.c_[L_imps]
        L_use= []
        for i in range(arr_imp.shape[1]):
            ci_low, ci_high =np.percentile(\
                                np.random.choice(arr_imp[:, i],\
                                                (1000, np.int32(np.ceil(.1*n_iterations)))\
                                                ).mean(1),\
                                [2.5, 97.5])
                                          
            if ci_low>0:
                L_use.append(i)  
        
        features = X_tr_c.columns[L_use]
        cat_features = np.intersect1d(cat_features, L_use)
        del X_tr_c
        gc.collect()
        
        return {'features':features, 'cat_features':cat_features}  
        
    def fit(self, X_tr, y_tr):
        
        X_tr_c = X_tr.copy()
        
        while True:

            d_sel_res = self.select_features(X_tr_c, y_tr, self.cv, self.scoring, self.params, self.seed)
            self.d_perm_imp_res= self.apply_perm_imp(d_sel_res, X_tr_c, y_tr, self.scoring, self.params, self.n_iterations, self.seed)

            feat2use= self.d_perm_imp_res['features']
            if len(feat2use)==X_tr_c.shape[1]:
                break
            else:
                X_tr_c = X_tr_c[feat2use]        
        del X_tr_c
        gc.collect()
        
        self.best_iteration = d_sel_res['best_iter']
        
        return self
    
    def transform(self, X):
        return X[self.d_perm_imp_res['features']] 

class CatBoostOptimizer():
    
    def __init__(self, scoring_func, best_iteration, seed, direction, n_trials):
        self.scoring_func = scoring_func        
        self.best_iteration = best_iteration
        self.seed = seed
        self.direction = direction
        self.n_trials = n_trials

    def objective(self, trial):
                
        params = {
#         'iterations':trial.suggest_int('iterations', 20, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.025, 0.25),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 31),
        'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
                }
        
        params['cat_features']= self.cat_features
        params['iterations'] = self.best_iteration
                
        model = cb.CatBoostClassifier(**params, random_seed=self.seed)
        model.fit(self.X_tr_c, self.y_tr_c, verbose=0, eval_set=(self.X_ho_c, self.y_ho_c))
        
        y_proba = model.predict_proba(self.X_ho_c)[:, 1]
        
        return self.scoring_func(self.y_ho_c, y_proba)
    
    def fit(self, X_tr, y_tr):
        
        self.cat_features = np.argwhere(X_tr.dtypes.values=='object').flatten()
        
        self.X_tr_c, self.X_ho_c, self.y_tr_c, self.y_ho_c = \
            train_test_split(X_tr, y_tr,
                             test_size = .1,
                             shuffle = True,
                             random_state =self.seed)
        
        sampler = TPESampler(seed=self.seed)
        study = optuna.create_study(direction=self.direction, sampler=sampler)
        study.optimize(self.objective, n_trials=self.n_trials)
        self.best_params = study.best_params
        
        del self.X_tr_c, self.X_ho_c, self.y_tr_c, self.y_ho_c
        gc.collect()
        
        return self
    
    def transform(self):
        return self.best_params        

### Признаки

<code>
    
import os, tqdm, json, pickle, gc
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from multiprocessing import Pool


def add_features4game(game_id):
    
    def add_features(date, team_id,start_ct, map_id, lineup,opp_lineup,  opponent_id,L_p_id, L_opp_p_id,
                     df_history, agg_keys, by_keys, prefix):
    
        d = {}

        d.update(df_history[agg_keys].add_prefix(f'{prefix}').add_suffix('__mean').mean().to_dict())
        d.update(df_history[agg_keys].add_prefix(f'{prefix}').add_suffix('__sum').sum().to_dict())

        if by_keys is not None:
            for by_key in by_keys:
                for by_value, subdf in df_history.groupby(by_key):
                    d.update(subdf[agg_keys].add_prefix(f'{prefix}{by_key}_{by_value}__').add_suffix('__mean').mean().to_dict())
                    d.update(subdf[agg_keys].add_prefix(f'{prefix}{by_key}_{by_value}__').add_suffix('__sum').sum().to_dict())


        return d

    def get_dicts4team(df_profile, date, team_id, start_ct, map_id, lineup,opp_lineup, opponent_id, L_p_id, L_opp_p_id):

        L_DF = [
            df_profile.query("""(date<@date)&(team_id==@team_id)"""),
            df_profile.query("""(date<@date)&(team_id==@team_id)&(map_id==@map_id)&(start_ct==@start_ct)"""),
            df_profile.query("""(date<@date)&(team_id==@team_id)&(lineup==@lineup)"""),
            df_profile.query("""(date<@date)&(team_id==@team_id)&(opponent_id==@opponent_id)"""),
        ]

        L_KEY = [
            'team', 'team_with_map_and_start', 'team_with_lineup', 'team_with_opponent'
        ]

        for i, p_id in enumerate(L_p_id):
            L_DF.append(df_profile.query("""(date<@date)&(team_id==@team_id)&(player_id==@p_id)"""))
            L_KEY.append(f'player{i+1}_in_team')
            L_DF.append(df_profile.query("""(date<@date)&(team_id!=@team_id)&(player_id==@p_id)"""))
            L_KEY.append(f'player{i+1}_not_in_team')
            for j, opp_p_id in enumerate(L_opp_p_id):
                L_DF.append(df_profile.query("""(date<@date)&(player_id in (@p_id, @opp_p_id))"""))
                L_KEY.append(f'player{i+1}_opp_player{j+1}')

        return dict(zip(L_KEY, L_DF))

    
    try:
    
        df_profile = pd.read_pickle('df_profile.pickle')
        df_game = df_profile.query('id==@game_id')

        date, map_id, serie_tier = df_game[['date', 'map_id', 'serie_tier']].iloc[0].values

        d_team_id2start_ct = dict(zip(df_game['team_id'], df_game['start_ct']))
        d_team_id2opponent_id = dict(zip(df_game['team_id'], df_game['opponent_id']))
        d_team_id2lineup = dict(zip(df_game['team_id'], df_game['lineup']))

        d_team_id2location = dict(zip(df_game['team_id'], df_game['team_location']))
        d_player_id2nationality = dict(zip(df_game['player_id'], df_game['player_nationality']))
        d_player_id2birthday = dict(zip(df_game['player_id'], df_game['player_birthday']))

        d_fs4gm = df_game[L_GLOBAL_KEYS].iloc[0].to_dict()

        for team_id, start_ct in d_team_id2start_ct.items():

            lineup = d_team_id2lineup[team_id]
            loc = d_team_id2location[team_id]        
            L_p_id = np.int32(np.array(lineup.split('-')))
            L_nat = [d_player_id2nationality[p_id] for p_id in L_p_id]
            L_bd = [d_player_id2birthday[p_id] for p_id in L_p_id]

            opponent_id = d_team_id2opponent_id[team_id]
            opp_lineup = d_team_id2lineup[opponent_id]
            opp_loc = d_team_id2location[opponent_id]        
            L_opp_p_id = np.int32(np.array(opp_lineup.split('-')))
            L_opp_nat = [d_player_id2nationality[p_id] for p_id in L_opp_p_id]
            L_opp_bd = [d_player_id2birthday[p_id] for p_id in L_opp_p_id]


            PREFIX = 'START_CT' if start_ct==1 else 'START_T'

            d_fs4gm[f'{PREFIX}__team_id'] = team_id
            d_fs4gm[f'{PREFIX}__team_location'] = loc
            d_fs4gm[f'{PREFIX}__team_lineup'] = lineup
            for i, (p_id, p_nat, p_bd) in enumerate(zip(L_p_id, L_nat, L_bd)):
                d_fs4gm[f'{PREFIX}__player{i+1}_id'] = p_id
                d_fs4gm[f'{PREFIX}__player{i+1}_nationality'] = p_nat
                d_fs4gm[f'{PREFIX}__player{i+1}_birthday'] = p_bd

            d_dicts4team = \
                get_dicts4team(
                    df_profile,
                    date,
                    team_id,
                    start_ct,
                    map_id,
                    lineup,
                    opp_lineup,
                    opponent_id,
                    L_p_id,
                    L_opp_p_id
                )        
            for suffix, df_history in d_dicts4team.items():    
                d_fs4gm.update(
                    add_features(
                        date,
                        team_id,
                        start_ct,
                        map_id,
                        lineup,
                        opp_lineup,
                        opponent_id,
                        L_p_id,
                        L_opp_p_id,
                        df_history,
                        L_AGG_KEYS,
                        L_BY_KEYS,
                        prefix=f'{PREFIX}__{suffix}__'
                    )
                )

        del d_dicts4team
        gc.collect()

        return d_fs4gm
    
    except:
        pass    

def reduce_mem_usage(series):
    try:
        col_type = series.dtype

        if col_type != object:
            c_min = series.min()
            c_max = series.max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    series = series.astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    series = series.astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    series = series.astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    series = series.astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    series = series.astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    series = series.astype(np.float32)
                else:
                    series = series.astype(np.float64)
        else:
            pass 
    except:
        pass
    
    return series



df_profile = pd.read_pickle('df_profile.pickle')

L_GLOBAL_KEYS = [
    
    'id', 'date', 
    'year', 'month', 'day', 'weekday', 'hour', 
    'map_id', 'serie_tier'
    
]

L_AGG_KEYS = [
    
    'win', 'maxround',
    
    'h1_win_count', 'h2_win_count',
    'h1_outcome_eliminated_count', 'h1_outcome_exploded_count',
    'h1_outcome_defused_count', 'h2_outcome_exploded_count',
    'h2_outcome_eliminated_count', 'h2_outcome_timeout_count',
    'h1_outcome_timeout_count',
    'h2_outcome_defused_count',
    
    'adr', 'assists', 'deaths', 'first_kills_diff',
    'flash_assists', 'headshots','k_d_diff', 'kast',
    'kills', 'rating'
    
]

L_BY_KEYS = [
    
    'year', 'month', 'day', 'weekday', 'hour',
    'serie_tier'
    
]


L_GAME_IDXS = np.unique(df_profile['id'])[-1000:]
L_BATCHES = np.array_split(L_GAME_IDXS, np.int32(np.ceil(len(L_GAME_IDXS)/10))) 
data = pd.DataFrame()

if __name__ =='__main__': 
    
    i=1
    for batch in L_BATCHES:
        

        print('> batch#{}/{}'.format(i, len(L_BATCHES)))
        with Pool(8) as p:
            arr = np.array(list(tqdm.tqdm(p.imap(add_features4game, batch), total=len(batch))))

        df =pd.DataFrame.from_records(arr[pd.notnull(arr)])
        print('> new_rows = {}'.format(df.shape))  
        data = data.append(df.apply(reduce_mem_usage))

        del df
        gc.collect()   

        data.to_pickle('df_features.pickle')        

        i+=1  
                    
------------------------------------------------------------------------------------------------------------------- 
> batch#1/100
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:48<00:00,  4.87s/it]
> n_rows: 10, n_keys: 295197                    
> batch#2/100
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:42<00:00,  4.30s/it]
> n_rows: 10, n_keys: 291809
-------------------------------------------------------------------------------------------------------------------                 
<code>

### 1. извлечение признаков и целевой переменной

In [2]:
PATH_TO_DATA = r'C:\Users\Sergey\anaconda3\Scripts\L_games_collection'

L_filenames = np.array(os.listdir(PATH_TO_DATA))
idx_argsorted = np.argsort([int(x.split('.')[0]) for x in L_filenames])
L_filenames = L_filenames[idx_argsorted]

In [3]:
ser_targets = pd.Series()
for fnm in tqdm.tqdm_notebook(L_filenames):  
        
    try:
        pth = os.path.join(PATH_TO_DATA, fnm)
        with open(pth, 'r') as f:
            d_rsp = json.load(f)
        df_rounds = pd.DataFrame.from_records(d_rsp['rounds'])
        start_ct_id = df_rounds.query('round==1')['ct'].iloc[0]
        winner_id = df_rounds['winner_team'].value_counts().idxmax()
        ser_targets.loc[d_rsp['id']] = int(start_ct_id==winner_id)
    except:
        pass 

  0%|          | 0/56435 [00:00<?, ?it/s]

In [4]:
PATH_TO_FEATURES = r'D:\features_17042022'

In [5]:
L_FILENAMES = np.array(os.listdir(PATH_TO_FEATURES))
L_FILENAMES = L_FILENAMES[np.argsort([int(x.split('_')[0][2:]) for x in L_FILENAMES])]

In [6]:
L_FILENAMES.shape

(386,)

In [7]:
# with open('L_selected_features.pickle', 'rb') as f:
#     L_selected_features = pickle.load(f)

In [42]:
L_date4game = []
for fnm in tqdm.tqdm_notebook(L_filenames): 
    try:
        pth = os.path.join(PATH_TO_DATA, fnm)
        with open(pth, 'r') as f:
            d_rsp = json.load(f)
        d = {'id':d_rsp['id'], 'date':parser.parse(d_rsp['begin_at'])}
        L_date4game.append(d)
    except:
        pass
df_date4game = pd.DataFrame.from_records(L_date4game)
df_date4game['year-month']= df_date4game['date'].dt.year*100+df_date4game['date'].dt.month

In [9]:
game_count_by_ym = df_date4game['year-month'].value_counts().sort_index()

In [10]:
game_count_by_ym[game_count_by_ym>=1000]

201911    1152
202002    1213
202003    1476
202004    1326
202005    1663
202006    1727
202007    1193
202008    1401
202009    1678
202010    1802
202011    1327
202012    1177
202101    1111
202102    1295
202103    2202
202104    1763
202105    2191
202106    1886
202107    1161
202108    1794
202109    2796
202110    2220
202111    2266
202112    1505
202201    1166
202202    1710
202203    2591
Name: year-month, dtype: int64

In [41]:
# df_features=pd.DataFrame()
for fnm in tqdm.tqdm_notebook(L_FILENAMES[279:]):
    pth = os.path.join(PATH_TO_FEATURES, fnm)
    df = pd.read_pickle(pth)
    df= df[df['date']>='2020-01-01']
    if len(df)==0:
        break
    else:        
        df_features = df_features.append(df)
        del df
        gc.collect()
# df_features = df_features.set_index('id')

In [14]:
df_features = df_features.set_index('id')

In [15]:
df_features.head()

,date,year,month,day,weekday,hour,map_id,serie_tier,START_T__team_id,START_T__team_location,...,START_CT__team_with_lineup__serie_tier_unranked__adr__sum,START_CT__team_with_lineup__serie_tier_unranked__assists__sum,START_CT__team_with_lineup__serie_tier_unranked__deaths__sum,START_CT__team_with_lineup__serie_tier_unranked__first_kills_diff__sum,START_CT__team_with_lineup__serie_tier_unranked__flash_assists__sum,START_CT__team_with_lineup__serie_tier_unranked__headshots__sum,START_CT__team_with_lineup__serie_tier_unranked__k_d_diff__sum,START_CT__team_with_lineup__serie_tier_unranked__kast__sum,START_CT__team_with_lineup__serie_tier_unranked__kills__sum,START_CT__team_with_lineup__serie_tier_unranked__rating__sum
id,,,,,,,,,,,,,,,,,,,,,
70608,2022-04-14 18:52:19+00:00,2022,4,14,3,18,8,c,3249,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70607,2022-04-14 17:42:54+00:00,2022,4,14,3,17,1,c,126507,GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70606,2022-04-14 20:03:32+00:00,2022,4,14,3,20,6,c,125775,DK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70605,2022-04-14 18:46:15+00:00,2022,4,14,3,18,20,c,125775,DK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70604,2022-04-14 17:47:44+00:00,2022,4,14,3,17,1,c,126755,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
idx2use = np.intersect1d(df_features.index, ser_targets.index)

y = ser_targets.loc[idx2use].dropna().astype(int)
X = df_features.loc[y.index].replace({None:np.nan})

assert (y.shape[0]!=0)&(y.shape[0]==X.shape[0])

del df_features, ser_targets
gc.collect()

55198

In [17]:
X.shape

(27868, 108108)

In [18]:
X.head()

,date,year,month,day,weekday,hour,map_id,serie_tier,START_T__team_id,START_T__team_location,...,START_CT__team_with_lineup__serie_tier_unranked__adr__sum,START_CT__team_with_lineup__serie_tier_unranked__assists__sum,START_CT__team_with_lineup__serie_tier_unranked__deaths__sum,START_CT__team_with_lineup__serie_tier_unranked__first_kills_diff__sum,START_CT__team_with_lineup__serie_tier_unranked__flash_assists__sum,START_CT__team_with_lineup__serie_tier_unranked__headshots__sum,START_CT__team_with_lineup__serie_tier_unranked__k_d_diff__sum,START_CT__team_with_lineup__serie_tier_unranked__kast__sum,START_CT__team_with_lineup__serie_tier_unranked__kills__sum,START_CT__team_with_lineup__serie_tier_unranked__rating__sum
17582,2020-04-08 13:46:45+00:00,2020,4,8,2,13,6,s,3240,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17584,2020-04-08 16:00:41+00:00,2020,4,8,2,16,1,s,3212,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17585,2020-04-08 17:18:43+00:00,2020,4,8,2,17,3,s,3216,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17586,2020-04-08 18:36:09+00:00,2020,4,8,2,18,6,s,3216,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17587,2020-04-08 20:08:42+00:00,2020,4,8,2,20,8,s,3209,DK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# L_glb_keys = ['year', 'month', 'day', 'weekday', 'hour', 'map_id']
# L_p_id_keys = [  
#     'START_T__team_id', 'START_CT__team_id',
#     'START_T__player1_id', 'START_T__player2_id', 'START_T__player3_id',
#     'START_T__player4_id', 'START_T__player5_id', 'START_CT__player1_id',
#     'START_CT__player2_id', 'START_CT__player3_id',
#     'START_CT__player4_id', 'START_CT__player5_id'
# ]
# L_bd_keys = [
#     'START_CT__player1_birthday', 'START_CT__player2_birthday',
#     'START_CT__player3_birthday', 'START_CT__player4_birthday',
#     'START_CT__player5_birthday', 'START_T__player1_birthday',
#     'START_T__player2_birthday', 'START_T__player3_birthday',
#     'START_T__player4_birthday', 'START_T__player5_birthday'
# ]
# L_loc_nat_keys = [    
#     'START_CT__team_location', 'START_T__team_location',
#     'START_CT__player1_nationality', 'START_CT__player2_nationality',
#     'START_CT__player3_nationality', 'START_CT__player4_nationality',
#     'START_CT__player5_nationality', 'START_T__player1_nationality',
#     'START_T__player2_nationality', 'START_T__player3_nationality',
#     'START_T__player4_nationality', 'START_T__player5_nationality'
# ]

# X = X.replace({None:np.nan})
# X[L_p_id_keys] = X[L_p_id_keys].fillna(-1).astype(int).astype(str)
# X[L_glb_keys] = X[L_glb_keys].fillna(-1).astype(int).astype(str)
# X['serie_tier'] = X['serie_tier'].fillna('default')

# df_bd = pd.concat([
    
#     X[L_bd_keys].astype('datetime64')\
#         .apply(lambda x: x.dt.year).fillna(-1).astype(int).astype(object).add_suffix('_year'),\
#     X[L_bd_keys].astype('datetime64')\
#         .apply(lambda x: x.dt.month).fillna(-1).astype(int).astype(object).add_suffix('_month'),\
#     X[L_bd_keys].astype('datetime64')\
#         .apply(lambda x: x.dt.day).fillna(-1).astype(int).astype(object).add_suffix('_day')\
# ], 1)
# X = pd.concat([X.drop(L_bd_keys, 1), df_bd], 1)
# X[L_loc_nat_keys] = X[L_loc_nat_keys].fillna('default')

# L_to_combo = L_glb_keys+L_p_id_keys+L_loc_nat_keys
# L_dfs = []
# for i in [2, 3]:
#     L_cmb =list(itertools.combinations(L_to_combo, i))
#     for cmb in tqdm.tqdm_notebook(L_cmb):
#         cmb = list(cmb)
#         new_key = '-'.join(cmb)
#         L_dfs.append(X[cmb].astype(str).apply(lambda x: '-'.join(x), axis = 1).to_frame(new_key))
# df_interactions = pd.concat(L_dfs, 1)
# X = pd.concat([X, df_interactions], 1)

# del df_interactions
# gc.collect()

In [21]:
X.to_pickle('X.pickle'), y.to_pickle('y.pickle')

(None, None)

In [22]:
X.fillna(-9999,inplace= True)

In [2]:
X, y = pd.read_pickle('X.pickle').fillna(-9999).drop('date', 1), pd.read_pickle('y.pickle')

In [3]:
X.shape

(27868, 108107)

In [4]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, shuffle = False, test_size = .1)
del X, y
gc.collect()

30

In [5]:
X_tr.head()

,year,month,day,weekday,hour,map_id,serie_tier,START_T__team_id,START_T__team_location,START_T__team_lineup,...,START_CT__team_with_lineup__serie_tier_unranked__adr__sum,START_CT__team_with_lineup__serie_tier_unranked__assists__sum,START_CT__team_with_lineup__serie_tier_unranked__deaths__sum,START_CT__team_with_lineup__serie_tier_unranked__first_kills_diff__sum,START_CT__team_with_lineup__serie_tier_unranked__flash_assists__sum,START_CT__team_with_lineup__serie_tier_unranked__headshots__sum,START_CT__team_with_lineup__serie_tier_unranked__k_d_diff__sum,START_CT__team_with_lineup__serie_tier_unranked__kast__sum,START_CT__team_with_lineup__serie_tier_unranked__kills__sum,START_CT__team_with_lineup__serie_tier_unranked__rating__sum
17582,2020,4,8,2,13,6,s,3240,DE,17524-17629-17663-17720-20397,...,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0
17584,2020,4,8,2,16,1,s,3212,US,17499-17522-17523-17526-20372,...,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0
17585,2020,4,8,2,17,3,s,3216,UA,17541-17542-17682-18002-19648,...,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0
17586,2020,4,8,2,18,6,s,3216,UA,17541-17542-17682-18002-19648,...,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0
17587,2020,4,8,2,20,8,s,3209,DK,17507-17508-17510-17511-17538,...,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0,-10000.0


### 2. пайплайн

    2.1. батчи признаков
    2.2. отбор признаков с помощью катбуста 
    2.3. объединение результатов батчей
    2.4. отбор признаков
    2.5. оптимизация гиперпараметров
    2.6. обычный блендинг

In [8]:
# итерации бустинга
CONST_PARAMS= {'iterations':100}
# итерации перестановки важностей
N_ITERATIONS = 100
# сид
SEED=13
# валидация
CV = StratifiedKFold(5, shuffle = True, random_state= SEED)
# метрика
SCORING ='roc_auc'

#### 2.1. батчи

In [9]:
L_all_features = X_tr.columns
batch_size = 20000
n_batches = np.int32(np.ceil(len(L_all_features) / batch_size))
L_batches = np.array_split(L_all_features, n_batches)

L_selected_features = []
for i, batch in tqdm.tqdm_notebook(enumerate(L_batches), total = n_batches):
    
    try:
        
        X_tr_batch = X_tr[batch]        
        cb_feat_sel = CatBoostFeatureSelector(CV, SCORING, CONST_PARAMS, N_ITERATIONS, SEED+i)
        cb_feat_sel.fit(X_tr_batch, y_tr)
        X_best_tr_batch = cb_feat_sel.transform(X_tr_batch)         
        L_selected_features.extend(X_best_tr_batch.columns.tolist())
        
        del X_tr_batch, X_best_tr_batch
        gc.collect()
        
    except:
        pass
    
X_tr_c, X_te_c = X_tr[L_selected_features], X_te[L_selected_features]
del X_tr, X_te
X_tr, X_te = X_tr_c, X_te_c
del X_tr_c, X_te_c
gc.collect()

with open('L_selected_features.pickle', 'wb') as f:
    pickle.dump(L_selected_features, f)

  0%|          | 0/6 [00:00<?, ?it/s]

> n_features = 18018
Learning rate set to 0.186312
0:	learn: 0.6841567	test: 0.6863625	best: 0.6863625 (0)	total: 2.83s	remaining: 4m 39s
1:	learn: 0.6782099	test: 0.6810127	best: 0.6810127 (1)	total: 6.35s	remaining: 5m 11s
2:	learn: 0.6735214	test: 0.6770516	best: 0.6770516 (2)	total: 9.72s	remaining: 5m 14s
3:	learn: 0.6697710	test: 0.6738581	best: 0.6738581 (3)	total: 11.7s	remaining: 4m 40s
4:	learn: 0.6665936	test: 0.6715787	best: 0.6715787 (4)	total: 13.3s	remaining: 4m 12s
5:	learn: 0.6637695	test: 0.6692576	best: 0.6692576 (5)	total: 14.9s	remaining: 3m 54s
6:	learn: 0.6617740	test: 0.6674867	best: 0.6674867 (6)	total: 16.6s	remaining: 3m 40s
7:	learn: 0.6603277	test: 0.6664994	best: 0.6664994 (7)	total: 18.2s	remaining: 3m 29s
8:	learn: 0.6587709	test: 0.6657264	best: 0.6657264 (8)	total: 19.7s	remaining: 3m 19s
9:	learn: 0.6567010	test: 0.6644831	best: 0.6644831 (9)	total: 21.2s	remaining: 3m 10s
10:	learn: 0.6556314	test: 0.6640221	best: 0.6640221 (10)	total: 22.7s	remainin

61:	learn: 0.5804829	total: 1.99s	remaining: 418ms
62:	learn: 0.5794031	total: 2.02s	remaining: 386ms
63:	learn: 0.5787557	total: 2.06s	remaining: 353ms
64:	learn: 0.5777108	total: 2.1s	remaining: 323ms
65:	learn: 0.5767307	total: 2.15s	remaining: 293ms
66:	learn: 0.5757469	total: 2.18s	remaining: 260ms
67:	learn: 0.5745800	total: 2.21s	remaining: 227ms
68:	learn: 0.5732823	total: 2.24s	remaining: 195ms
69:	learn: 0.5720305	total: 2.27s	remaining: 162ms
70:	learn: 0.5708176	total: 2.31s	remaining: 130ms
71:	learn: 0.5701864	total: 2.34s	remaining: 97.6ms
72:	learn: 0.5693952	total: 2.37s	remaining: 65ms
73:	learn: 0.5688730	total: 2.4s	remaining: 32.4ms
74:	learn: 0.5675488	total: 2.43s	remaining: 0us


  0%|          | 0/100 [00:00<?, ?it/s]

> n_features = 39
Learning rate set to 0.186312
0:	learn: 0.6836099	test: 0.6844452	best: 0.6844452 (0)	total: 28.3ms	remaining: 2.8s
1:	learn: 0.6765096	test: 0.6774191	best: 0.6774191 (1)	total: 76.8ms	remaining: 3.76s
2:	learn: 0.6718630	test: 0.6730717	best: 0.6730717 (2)	total: 124ms	remaining: 4.02s
3:	learn: 0.6682335	test: 0.6696803	best: 0.6696803 (3)	total: 170ms	remaining: 4.09s
4:	learn: 0.6654959	test: 0.6677154	best: 0.6677154 (4)	total: 202ms	remaining: 3.84s
5:	learn: 0.6632006	test: 0.6658633	best: 0.6658633 (5)	total: 236ms	remaining: 3.69s
6:	learn: 0.6614803	test: 0.6646343	best: 0.6646343 (6)	total: 266ms	remaining: 3.53s
7:	learn: 0.6599410	test: 0.6633963	best: 0.6633963 (7)	total: 308ms	remaining: 3.54s
8:	learn: 0.6580028	test: 0.6616724	best: 0.6616724 (8)	total: 355ms	remaining: 3.59s
9:	learn: 0.6565140	test: 0.6607543	best: 0.6607543 (9)	total: 386ms	remaining: 3.47s
10:	learn: 0.6548085	test: 0.6601837	best: 0.6601837 (10)	total: 417ms	remaining: 3.38s
11:

...

> n_features = 25
Learning rate set to 0.186312
0:	learn: 0.6849043	test: 0.6854055	best: 0.6854055 (0)	total: 37.1ms	remaining: 3.67s
1:	learn: 0.6790306	test: 0.6795245	best: 0.6795245 (1)	total: 45.5ms	remaining: 2.23s
2:	learn: 0.6745612	test: 0.6754595	best: 0.6754595 (2)	total: 54.1ms	remaining: 1.75s
3:	learn: 0.6710112	test: 0.6719399	best: 0.6719399 (3)	total: 62.5ms	remaining: 1.5s
4:	learn: 0.6677078	test: 0.6684168	best: 0.6684168 (4)	total: 70.5ms	remaining: 1.34s
5:	learn: 0.6645449	test: 0.6656422	best: 0.6656422 (5)	total: 78.6ms	remaining: 1.23s
6:	learn: 0.6621591	test: 0.6634332	best: 0.6634332 (6)	total: 86.3ms	remaining: 1.15s
7:	learn: 0.6601934	test: 0.6621112	best: 0.6621112 (7)	total: 94.5ms	remaining: 1.09s
8:	learn: 0.6579958	test: 0.6601966	best: 0.6601966 (8)	total: 102ms	remaining: 1.03s
9:	learn: 0.6554609	test: 0.6580506	best: 0.6580506 (9)	total: 110ms	remaining: 990ms
10:	learn: 0.6539350	test: 0.6570122	best: 0.6570122 (10)	total: 118ms	remaining: 955

  0%|          | 0/100 [00:00<?, ?it/s]

> n_features = 24
Learning rate set to 0.186312
0:	learn: 0.6853130	test: 0.6856343	best: 0.6856343 (0)	total: 32.5ms	remaining: 3.21s
1:	learn: 0.6792969	test: 0.6801119	best: 0.6801119 (1)	total: 41.3ms	remaining: 2.02s
2:	learn: 0.6749148	test: 0.6754123	best: 0.6754123 (2)	total: 49.8ms	remaining: 1.61s
3:	learn: 0.6707908	test: 0.6716102	best: 0.6716102 (3)	total: 58.5ms	remaining: 1.4s
4:	learn: 0.6668396	test: 0.6678384	best: 0.6678384 (4)	total: 67.2ms	remaining: 1.28s
5:	learn: 0.6636930	test: 0.6643727	best: 0.6643727 (5)	total: 76ms	remaining: 1.19s
6:	learn: 0.6616834	test: 0.6624180	best: 0.6624180 (6)	total: 84ms	remaining: 1.11s
7:	learn: 0.6595578	test: 0.6610954	best: 0.6610954 (7)	total: 92.1ms	remaining: 1.06s
8:	learn: 0.6573515	test: 0.6595670	best: 0.6595670 (8)	total: 100ms	remaining: 1.02s
9:	learn: 0.6554410	test: 0.6573982	best: 0.6573982 (9)	total: 108ms	remaining: 974ms
10:	learn: 0.6536670	test: 0.6560969	best: 0.6560969 (10)	total: 116ms	remaining: 936ms
1

28:	learn: 0.6202816	total: 244ms	remaining: 395ms
29:	learn: 0.6192120	total: 251ms	remaining: 385ms
30:	learn: 0.6180319	total: 259ms	remaining: 376ms
31:	learn: 0.6171186	total: 266ms	remaining: 366ms
32:	learn: 0.6165815	total: 274ms	remaining: 357ms
33:	learn: 0.6153284	total: 282ms	remaining: 348ms
34:	learn: 0.6142601	total: 289ms	remaining: 338ms
35:	learn: 0.6132571	total: 296ms	remaining: 329ms
36:	learn: 0.6123308	total: 304ms	remaining: 320ms
37:	learn: 0.6113305	total: 311ms	remaining: 311ms
38:	learn: 0.6103687	total: 318ms	remaining: 302ms
39:	learn: 0.6089541	total: 326ms	remaining: 293ms
40:	learn: 0.6075309	total: 333ms	remaining: 285ms
41:	learn: 0.6067427	total: 341ms	remaining: 276ms
42:	learn: 0.6056534	total: 348ms	remaining: 267ms
43:	learn: 0.6042068	total: 356ms	remaining: 259ms
44:	learn: 0.6031988	total: 363ms	remaining: 250ms
45:	learn: 0.6020204	total: 370ms	remaining: 242ms
46:	learn: 0.6009941	total: 378ms	remaining: 233ms
47:	learn: 0.6001421	total: 385

  0%|          | 0/100 [00:00<?, ?it/s]

#### 2.3. объединение признаков

In [10]:
cb_feat_sel = CatBoostFeatureSelector(CV, SCORING, CONST_PARAMS, N_ITERATIONS, SEED)

cb_feat_sel.fit(X_tr, y_tr)

X_best_tr = cb_feat_sel.transform(X_tr)
del X_tr
X_best_te = cb_feat_sel.transform(X_te)
del X_te
gc.collect()

best_iteration = cb_feat_sel.best_iteration

> n_features = 124
Learning rate set to 0.186312
0:	learn: 0.6819283	test: 0.6832969	best: 0.6832969 (0)	total: 43.6ms	remaining: 4.32s
1:	learn: 0.6730502	test: 0.6750774	best: 0.6750774 (1)	total: 82.3ms	remaining: 4.03s
2:	learn: 0.6667788	test: 0.6701215	best: 0.6701215 (2)	total: 119ms	remaining: 3.84s
3:	learn: 0.6613791	test: 0.6647401	best: 0.6647401 (3)	total: 165ms	remaining: 3.96s
4:	learn: 0.6565652	test: 0.6605470	best: 0.6605470 (4)	total: 222ms	remaining: 4.22s
5:	learn: 0.6522538	test: 0.6562047	best: 0.6562047 (5)	total: 263ms	remaining: 4.12s
6:	learn: 0.6485405	test: 0.6535279	best: 0.6535279 (6)	total: 304ms	remaining: 4.04s
7:	learn: 0.6454082	test: 0.6513097	best: 0.6513097 (7)	total: 343ms	remaining: 3.95s
8:	learn: 0.6420648	test: 0.6487016	best: 0.6487016 (8)	total: 410ms	remaining: 4.14s
9:	learn: 0.6388523	test: 0.6460482	best: 0.6460482 (9)	total: 450ms	remaining: 4.05s
10:	learn: 0.6358695	test: 0.6439379	best: 0.6439379 (10)	total: 492ms	remaining: 3.98s
1

  0%|          | 0/100 [00:00<?, ?it/s]

> n_features = 51
Learning rate set to 0.186312
0:	learn: 0.6816425	test: 0.6817589	best: 0.6817589 (0)	total: 32.4ms	remaining: 3.2s
1:	learn: 0.6733933	test: 0.6745682	best: 0.6745682 (1)	total: 63.3ms	remaining: 3.1s
2:	learn: 0.6670963	test: 0.6688581	best: 0.6688581 (2)	total: 101ms	remaining: 3.25s
3:	learn: 0.6612156	test: 0.6630756	best: 0.6630756 (3)	total: 150ms	remaining: 3.59s
4:	learn: 0.6575630	test: 0.6602463	best: 0.6602463 (4)	total: 179ms	remaining: 3.41s
5:	learn: 0.6515865	test: 0.6554263	best: 0.6554263 (5)	total: 213ms	remaining: 3.34s
6:	learn: 0.6484863	test: 0.6531571	best: 0.6531571 (6)	total: 249ms	remaining: 3.31s
7:	learn: 0.6446561	test: 0.6506285	best: 0.6506285 (7)	total: 281ms	remaining: 3.23s
8:	learn: 0.6410378	test: 0.6487830	best: 0.6487830 (8)	total: 327ms	remaining: 3.31s
9:	learn: 0.6380700	test: 0.6473588	best: 0.6473588 (9)	total: 363ms	remaining: 3.27s
10:	learn: 0.6359050	test: 0.6460766	best: 0.6460766 (10)	total: 394ms	remaining: 3.19s
11:	

  0%|          | 0/100 [00:00<?, ?it/s]

> n_features = 38
Learning rate set to 0.186312
0:	learn: 0.6808816	test: 0.6818022	best: 0.6818022 (0)	total: 30.2ms	remaining: 2.99s
1:	learn: 0.6719218	test: 0.6737861	best: 0.6737861 (1)	total: 80.8ms	remaining: 3.96s
2:	learn: 0.6650281	test: 0.6676715	best: 0.6676715 (2)	total: 129ms	remaining: 4.16s
3:	learn: 0.6593362	test: 0.6627642	best: 0.6627642 (3)	total: 162ms	remaining: 3.89s
4:	learn: 0.6536123	test: 0.6586339	best: 0.6586339 (4)	total: 197ms	remaining: 3.74s
5:	learn: 0.6493036	test: 0.6543356	best: 0.6543356 (5)	total: 233ms	remaining: 3.66s
6:	learn: 0.6462602	test: 0.6519269	best: 0.6519269 (6)	total: 283ms	remaining: 3.76s
7:	learn: 0.6426634	test: 0.6497730	best: 0.6497730 (7)	total: 331ms	remaining: 3.8s
8:	learn: 0.6401143	test: 0.6481191	best: 0.6481191 (8)	total: 377ms	remaining: 3.81s
9:	learn: 0.6376122	test: 0.6465232	best: 0.6465232 (9)	total: 425ms	remaining: 3.83s
10:	learn: 0.6359099	test: 0.6458426	best: 0.6458426 (10)	total: 459ms	remaining: 3.71s
11:

...

50:	learn: 0.5646070	total: 1.93s	remaining: 37.9ms
51:	learn: 0.5631021	total: 1.96s	remaining: 0us


  0%|          | 0/100 [00:00<?, ?it/s]

> n_features = 25
Learning rate set to 0.186312
0:	learn: 0.6812683	test: 0.6824282	best: 0.6824282 (0)	total: 48.2ms	remaining: 4.77s
1:	learn: 0.6719732	test: 0.6752003	best: 0.6752003 (1)	total: 93.5ms	remaining: 4.58s
2:	learn: 0.6649735	test: 0.6686512	best: 0.6686512 (2)	total: 123ms	remaining: 3.98s
3:	learn: 0.6597389	test: 0.6643751	best: 0.6643751 (3)	total: 154ms	remaining: 3.7s
4:	learn: 0.6545224	test: 0.6598528	best: 0.6598528 (4)	total: 186ms	remaining: 3.54s
5:	learn: 0.6491746	test: 0.6552930	best: 0.6552930 (5)	total: 218ms	remaining: 3.41s
6:	learn: 0.6458229	test: 0.6523305	best: 0.6523305 (6)	total: 249ms	remaining: 3.31s
7:	learn: 0.6426424	test: 0.6497146	best: 0.6497146 (7)	total: 282ms	remaining: 3.24s
8:	learn: 0.6407529	test: 0.6487292	best: 0.6487292 (8)	total: 313ms	remaining: 3.17s
9:	learn: 0.6384967	test: 0.6473881	best: 0.6473881 (9)	total: 356ms	remaining: 3.21s
10:	learn: 0.6368476	test: 0.6465626	best: 0.6465626 (10)	total: 388ms	remaining: 3.13s
11:

37:	learn: 0.5812971	total: 1.28s	remaining: 743ms
38:	learn: 0.5802982	total: 1.32s	remaining: 709ms
39:	learn: 0.5790302	total: 1.36s	remaining: 679ms
40:	learn: 0.5784018	total: 1.39s	remaining: 644ms
41:	learn: 0.5776955	total: 1.42s	remaining: 608ms
42:	learn: 0.5770898	total: 1.45s	remaining: 573ms
43:	learn: 0.5760556	total: 1.49s	remaining: 541ms
44:	learn: 0.5749185	total: 1.52s	remaining: 507ms
45:	learn: 0.5738810	total: 1.56s	remaining: 474ms
46:	learn: 0.5727400	total: 1.58s	remaining: 439ms
47:	learn: 0.5713094	total: 1.62s	remaining: 405ms
48:	learn: 0.5704337	total: 1.65s	remaining: 370ms
49:	learn: 0.5691688	total: 1.68s	remaining: 336ms
50:	learn: 0.5675610	total: 1.71s	remaining: 302ms
51:	learn: 0.5667682	total: 1.76s	remaining: 270ms
52:	learn: 0.5652334	total: 1.78s	remaining: 236ms
53:	learn: 0.5640070	total: 1.83s	remaining: 204ms
54:	learn: 0.5630682	total: 1.86s	remaining: 170ms
55:	learn: 0.5620444	total: 1.9s	remaining: 135ms
56:	learn: 0.5609664	total: 1.93

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
X_best_tr.head()

,START_T__player5_id,START_T__player5_birthday,START_T__team_with_map_and_start__year_2020__adr__mean,START_T__team_with_map_and_start__day_10__deaths__sum,START_T__team_with_lineup__rating__mean,START_T__team_with_lineup__k_d_diff__sum,START_T__team_with_opponent__win__mean,START_T__team_with_opponent__rating__mean,START_T__team_with_opponent__k_d_diff__sum,START_T__player2_not_in_team__rating__sum,...,START_CT__player5_birthday,START_CT__team__weekday_5__win__sum,START_CT__team_with_map_and_start__k_d_diff__mean,START_CT__team_with_lineup__k_d_diff__mean,START_CT__player2_not_in_team__day_30__adr__mean,START_CT__player3_in_team__h1_win_count__mean,START_CT__player5_in_team__serie_tier_a__maxround__sum,START_CT__player5_not_in_team__day_18__k_d_diff__mean,START_CT__player5_not_in_team__day_26__headshots__sum,START_T__team_with_lineup__weekday_0__win__mean
17582,20397,2002-07-18,77.5000,-10000.0,1.068359,1462.0,0.538574,1.002930,221.0,0.000000,...,2002-06-17,300.0,-1.023438,0.688477,65.8125,7.683594,2088.0,-10000.0,-10000.0,0.600098
17584,20372,2001-02-14,78.4375,91.0,1.085938,572.0,0.424316,1.015625,-15.0,62.718750,...,1999-11-24,330.0,1.713867,1.355469,-10000.0000,7.996094,-10000.0,-10000.0,56.0,0.611328
17585,19648,1999-11-24,75.9375,-10000.0,1.105469,367.0,0.558594,1.022461,-20.0,0.000000,...,2001-02-14,360.0,0.420898,1.150391,75.3125,7.917969,556.0,-10000.0,-10000.0,0.666504
17586,19648,1999-11-24,78.5000,-10000.0,1.104492,372.0,0.571289,1.023438,-15.0,0.000000,...,2001-02-14,360.0,0.114258,1.125977,75.3125,7.925781,556.0,-10000.0,-10000.0,0.666504
17587,17538,1998-03-05,-10000.0000,100.0,1.130859,5280.0,-10000.000000,-10000.000000,0.0,10.796875,...,1997-08-21,30.0,2.000000,-0.063354,80.4375,7.234375,-10000.0,-10000.0,-10000.0,0.759766


In [12]:
X_best_tr.to_pickle('X_best_tr.pickle')
X_best_te.to_pickle('X_best_te.pickle')

#### 2.5. оптимизация гиперпараметров

In [13]:
cb_opt = CatBoostOptimizer(
            scoring_func= lambda y, y_proba: roc_auc_score(y, y_proba),
            best_iteration=best_iteration,
            seed=SEED, 
            direction='maximize',
            n_trials=50
)

cb_opt.fit(X_best_tr, y_tr)

best_params = cb_opt.transform()
best_params['verbose'] = 0

[I 2022-04-18 06:46:26,265] A new study created in memory with name: no-name-a94b66be-4024-4f01-80a9-dd4570163b41
[I 2022-04-18 06:46:28,311] Trial 0 finished with value: 0.6894355922100337 and parameters: {'learning_rate': 0.19998304237910955, 'depth': 5, 'l2_leaf_reg': 25.728355979841055, 'boosting_type': 'Plain'}. Best is trial 0 with value: 0.6894355922100337.
[I 2022-04-18 06:46:32,999] Trial 1 finished with value: 0.68857102186432 and parameters: {'learning_rate': 0.12702608066889526, 'depth': 9, 'l2_leaf_reg': 24.2657954381454, 'boosting_type': 'Plain'}. Best is trial 0 with value: 0.6894355922100337.
[I 2022-04-18 06:46:35,969] Trial 2 finished with value: 0.6843101294218277 and parameters: {'learning_rate': 0.0328832179228234, 'depth': 5, 'l2_leaf_reg': 2.7553747564622424, 'boosting_type': 'Ordered'}. Best is trial 0 with value: 0.6894355922100337.
[I 2022-04-18 06:46:38,106] Trial 3 finished with value: 0.6885700686445342 and parameters: {'learning_rate': 0.17796578910507183,

[I 2022-04-18 06:48:32,074] Trial 31 finished with value: 0.6872050579112795 and parameters: {'learning_rate': 0.1092872999809807, 'depth': 5, 'l2_leaf_reg': 5.525986881110049, 'boosting_type': 'Ordered'}. Best is trial 26 with value: 0.6896113023905481.
[I 2022-04-18 06:48:35,667] Trial 32 finished with value: 0.6868333021948203 and parameters: {'learning_rate': 0.07128320511910188, 'depth': 6, 'l2_leaf_reg': 5.989593935180594, 'boosting_type': 'Ordered'}. Best is trial 26 with value: 0.6896113023905481.
[I 2022-04-18 06:48:38,517] Trial 33 finished with value: 0.6851070211627501 and parameters: {'learning_rate': 0.116695825288858, 'depth': 5, 'l2_leaf_reg': 2.9873861994052673, 'boosting_type': 'Ordered'}. Best is trial 26 with value: 0.6896113023905481.
[I 2022-04-18 06:48:40,825] Trial 34 finished with value: 0.684976112312168 and parameters: {'learning_rate': 0.1401906091667538, 'depth': 4, 'l2_leaf_reg': 12.671899933702084, 'boosting_type': 'Ordered'}. Best is trial 26 with value:

In [17]:
d_cb_tuned_scores=get_cb_scores(best_params,
                                X_best_tr, X_best_te,
                                y_tr, y_te)
d_cb_scores=get_cb_scores({'iterations':best_iteration, 'verbose':0},
                          X_best_tr, X_best_te,
                          y_tr, y_te)

In [ ]:
print('> auc(untuned): {}'.format(d_cb_scores))

In [ ]:
print('> auc(tuned): {}'.format(d_cb_tuned_scores))

#### 2.6. обычный блендинг

In [85]:
# cat_features = np.argwhere(X_best_tr.dtypes.values=='object').flatten()

# n_iterations = 10

# L_te_proba = []
# L_models = []

# model = cb.CatBoostClassifier(**best_params)  
# L_models.append(model)
# model.fit(X_best_tr, y_tr)
# L_te_proba.append(model.predict_proba(X_best_te)[:, 1])

# for seed in tqdm.tqdm_notebook(range(n_iterations)):
    
#     np.random.seed(SEED+seed)
    
#     best_params_c = best_params.copy()
#     best_params_c['depth'] + np.random.choice(np.arange(-2, 2))
#     best_params_c['l2_leaf_reg'] + np.random.choice(np.arange(-2, 2))
#     best_params_c['random_state'] =SEED+seed
#     best_params_c['cat_features']= cat_features
    
#     model = cb.CatBoostClassifier(**best_params_c)
#     L_models.append(model)
    
#     model.fit(X_best_tr, y_tr)    
#     L_te_proba.append(model.predict_proba(X_best_te)[:, 1])

In [87]:
L_proba_idxs = np.arange(len(L_te_proba))
L_te_proba = np.array(L_te_proba)
L_models = np.array(L_models)

L_blend_scores =[]
L_blend_combos= []
for i in range(1, len(L_proba_idxs)+1):
    for cmb in itertools.combinations(L_proba_idxs, i):
        cmb = list(cmb)
        auc_blend = roc_auc_score(y_te, L_te_proba[cmb].T.mean(1))
        L_blend_scores.append(auc_blend)
        L_blend_combos.append(cmb)
        
best_combo_idx = np.argmax(L_blend_scores)
best_score = L_blend_scores[best_combo_idx]
best_models = L_models[L_blend_combos[best_combo_idx]]

In [90]:
print('> auc(blended): {}'.format(best_score))

for m in best_models:
    print('> model: {}'.format(m.get_params()))

> auc(blended): 0.6818760299648701
> model: {'learning_rate': 0.10469925677185019, 'depth': 5, 'l2_leaf_reg': 5.851196188170178, 'verbose': 0, 'boosting_type': 'Ordered', 'cat_features': array([ 0,  1, 13, 14, 15], dtype=int64)}
> model: {'learning_rate': 0.10469925677185019, 'depth': 5, 'l2_leaf_reg': 5.851196188170178, 'verbose': 0, 'boosting_type': 'Ordered', 'random_state': 13, 'cat_features': array([ 0,  1, 13, 14, 15], dtype=int64)}
> model: {'learning_rate': 0.10469925677185019, 'depth': 5, 'l2_leaf_reg': 5.851196188170178, 'verbose': 0, 'boosting_type': 'Ordered', 'random_state': 16, 'cat_features': array([ 0,  1, 13, 14, 15], dtype=int64)}
> model: {'learning_rate': 0.10469925677185019, 'depth': 5, 'l2_leaf_reg': 5.851196188170178, 'verbose': 0, 'boosting_type': 'Ordered', 'random_state': 19, 'cat_features': array([ 0,  1, 13, 14, 15], dtype=int64)}


In [91]:
X = pd.concat([X_best_tr, X_best_te])
y = pd.concat([y_tr, y_te])

assert X.shape[0] == y.shape[0]

X = X.sort_index()
y = y.loc[X.index]

In [92]:
with open('best_models.pickle', 'wb') as f:
    pickle.dump(best_models, f)
with open('best_data.pickle', 'wb') as f:
    pickle.dump((X, y), f)

In [3]:
with open('best_models.pickle', 'rb') as f:
    best_models = pickle.load(f)
with open('best_data.pickle', 'rb') as f:
    X, y = pickle.load(f)

In [4]:
df_data4model = pd.read_pickle('X.pickle')[['map_id', 'START_CT__team_id', 'START_T__team_id']]

In [5]:
L_all_features = X.columns

L_fs4ct, L_fs4t = [], []
for f in L_all_features:
    if 'START_CT' in f:
        L_fs4ct.append(f)
    else:
        L_fs4t.append(f) 

In [6]:
L_ct = []
for (map_id, start_ct_id), subdf in df_data4model.groupby(['map_id', 'START_CT__team_id']):
    idx4ct = subdf.index[-1]
    d_fs = {'map_id':map_id, 'team_id':start_ct_id, 'n_games':subdf.shape[0], 'accuracy':(y.loc[subdf.index]==1).mean()}
    d_fs['features'] = X.loc[idx4ct, L_fs4ct].to_dict()
    L_ct.append(d_fs)
df_ct = pd.DataFrame.from_records(L_ct)

L_t = []
for (map_id, start_t_id), subdf in df_data4model.groupby(['map_id', 'START_T__team_id']):
    idx4t = subdf.index[-1]
    d_fs = {'map_id':map_id, 'team_id':start_t_id, 'n_games':subdf.shape[0], 'accuracy':(y.loc[subdf.index]==0).mean()}
    d_fs['features'] = X.loc[idx4t, L_fs4t].to_dict()
    L_t.append(d_fs)   
df_t = pd.DataFrame.from_records(L_t)    

In [7]:
df_ct.head()

,map_id,team_id,n_games,accuracy,features
0,1,124522,42,0.595238,"{'START_CT__player4_id': '29413', 'START_CT__p..."
1,1,124523,42,0.523810,"{'START_CT__player4_id': '25660', 'START_CT__p..."
2,1,124524,8,0.375000,"{'START_CT__player4_id': '20932', 'START_CT__p..."
3,1,124527,22,0.545455,"{'START_CT__player4_id': '19659', 'START_CT__p..."
4,1,124530,32,0.625000,"{'START_CT__player4_id': '24811', 'START_CT__p..."


In [8]:
df_games = pd.read_pickle('X.pickle')[['date', 'START_CT__team_id', 'START_T__team_id']]
df_games = df_games[df_games['date'].dt.year==2022]

In [9]:
start_ct2use = df_games['START_CT__team_id'].unique()
start_t2use = df_games['START_T__team_id'].unique()

In [10]:
df_ct = df_ct.query('team_id in @start_ct2use')
df_t = df_t.query('team_id in @start_t2use')

In [11]:
d_map_id2name = {}
d_team_id2name= {}
for fnm in tqdm.tqdm_notebook(L_filenames):  
        
    try:
        pth = os.path.join(PATH_TO_DATA, fnm)
        with open(pth, 'r') as f:
            d_rsp = json.load(f)
        d_map_id2name[d_rsp['map']['id']] = str.lower(d_rsp['map']['name']).strip()
        for t in d_rsp['teams']:
            d_team_id2name[t['id']]= str.lower(t['name']).strip()
    except:
        pass
            

  0%|          | 0/56435 [00:00<?, ?it/s]

In [ ]:
L_answers= []

for (map_id, team_id), subdf_ct in tqdm.tqdm_notebook(df_ct.groupby(['map_id', 'team_id'])):
    
    ct_acc = subdf_ct[['n_games','accuracy']].iloc[0].to_dict()
    subdf_t = df_t.query('(map_id==@map_id)&(team_id!=@team_id)')
    
    d_fs4gm = subdf_ct['features'].iloc[0]
    
    for i in range(len(subdf_t)):
        r= subdf_t.iloc[i]
        t_acc= r[['n_games','accuracy']].to_dict()
        d_fs4gm.update(r['features'])
        x_new = pd.DataFrame.from_records([d_fs4gm])[L_all_features]
        proba= np.mean([m.predict_proba(x_new)[:, 1][0] for m in best_models])
        
        _d= {'map':d_map_id2name[int(map_id)],
                          'start_ct':d_team_id2name[int(team_id)],
                          'start_ct_n_games':ct_acc['n_games'],
                          'start_ct_accuracy':round(ct_acc['accuracy'],3),
                          'start_t':d_team_id2name[int(r['team_id'])],
                          'start_t_n_games':t_acc['n_games'],
                          'start_t_accuracy':round(t_acc['accuracy'], 3),
                          'start_ct_win_proba':proba,
                          'start_t_win_proba':1-proba}
        L_answers.append(_d)
    
df_answers=pd.DataFrame.from_records(L_answers)

  0%|          | 0/1770 [00:00<?, ?it/s]

In [ ]:
df_answers.to_pickle('df_answers.pickle')

In [ ]:
df_answers.head()

In [ ]:
L_START_CT_UNIQUE= df_answers['start_ct'].unique()
L_START_T_UNIQUE= df_answers['start_t'].unique()
L_MAP_UNIQUE = df_answers['map'].unique()

In [ ]:
def get_answer(map_name, start_ct, start_t):
    
    df_v1= df_answers.query('(start_ct==@start_ct)&(start_t==@start_t)')
    df_v2=df_answers.query('(start_t==@start_ct)&(start_ct==@start_t)')

    # на основе шансов для пары команд на всех картах за каждую сторону
    start_ct_chances = np.mean([df_v1['start_ct_win_proba'].mean(), df_v2['start_t_win_proba'].mean()])
    start_t_chances=  np.mean([df_v1['start_t_win_proba'].mean(), df_v2['start_ct_win_proba'].mean()])

    # на основе шансов для карты+стороны
    df_v1_map = df_v1.query('map==@map')
    ct_on_map_proba = df_v1_map['start_ct_win_proba'].iloc[0]
    ct_err = df_v1_map['start_ct_accuracy'].iloc[0]*ct_on_map_proba
    t_on_map_proba = df_v1_map['start_t_win_proba'].iloc[0]
    t_err = df_v1_map['start_t_accuracy'].iloc[0]*t_on_map_proba

    # на основе шансов против остальных команд на карте за сторону
    p_ct, p_t = df_answers.query('(start_ct==@start_ct)&(start_t!=@start_t)')['start_ct_win_proba'].mean(),\
                df_answers.query('(start_ct!=@start_ct)&(start_t==@start_t)')['start_t_win_proba'].mean()
    start_ct_vs_others, start_t_vs_others = p_ct/(p_ct+p_t), p_t/(p_ct+p_t)

    ct_answer = {
        'start_ct_on_map_proba_low':ct_on_map_proba-ct_err,
        'start_ct_on_map_proba':ct_on_map_proba,
        'start_ct_on_map_proba_high':ct_on_map_proba+ct_err,
        'start_ct_all_map_all_start_proba':start_ct_chances, 
        'start_ct_vs_others_proba':start_ct_vs_others
    }
    t_answer = {
        'start_t_on_map_proba_low':t_on_map_proba-t_err,
        'start_t_on_map_proba':t_on_map_proba,
        'start_t_on_map_proba_high':t_on_map_proba+t_err,
        'start_t_all_map_all_start_proba':start_t_chances,
        'start_t_vs_others_proba':start_t_vs_others
    }
    
    d = {}
    
    d.update(ct_answer)
    d.update(t_answer)
    
    return d

In [ ]:
L_answers_v2 = []
for map_name in tqdm.tqdm_notebook(L_MAP_UNIQUE):
    for start_ct in tqdm.tqdm_notebook(L_START_CT_UNIQUE):
        for start_t in L_START_T_UNIQUE:
            try:
                d_info ={'map':map_name, 'start_ct':start_ct, 'start_t':start_t}
                d = get_answer(map_name, start_ct, start_t)
                d_info.update(d)
                L_answers_v2.append(d_info)
            except:
                pass
    

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/353 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

In [257]:
d

{'start_ct_on_map_proba_low': 0.15632114707848627,
 'start_ct_on_map_proba': 0.385978140934534,
 'start_ct_on_map_proba_high': 0.6156351347905817,
 'start_ct_all_map_all_start_proba': 0.4583683749970316,
 'start_ct_vs_others_proba': 0.4441202731568624,
 'start_t_on_map_proba_low': 0.2456087436261864,
 'start_t_on_map_proba': 0.614021859065466,
 'start_t_on_map_proba_high': 0.9824349745047456,
 'start_t_all_map_all_start_proba': 0.5416316250029684,
 'start_t_vs_others_proba': 0.5558797268431376}

0.5954798610386722